In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoAlertPresentException

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as BS
from datetime import datetime
from random import *

import pandas as pd
import time
import pyperclip
import sys, os
import random
import re
import math
import ast


def find_css(css_selector, browser):
    return browser.find_element(By.CSS_SELECTOR, css_selector)
def finds_css(css_selector, browser):
    return browser.find_elements(By.CSS_SELECTOR, css_selector)

def find_xpath(xpath, browser):
    return browser.find_element(By.XPATH, xpath)
def finds_xpath(xpath, browser):
    return browser.find_elements(By.XPATH, xpath)

def find_id(e_id, browser):
    return browser.find_element(By.ID, e_id)

def find_className(cn, browser):
    return browser.find_element(By.CLASS_NAME, cn)
def finds_className(cn , browser):
    return browser.find_elements(By.CLASS_NAME, cn)

def find_linktext(lt, browser):
    return browser.find_element(By.LINK_TEXT, lt)

def find_name(name, browser):
    return browser.find_element(By.NAME, name)
def finds_name(name, browser):
    return browser.find_elements(By.NAME, name)

def resource_path(relative_path):
    """ Get absolute path to resource, works for dev and for PyInstaller """
    base_path = getattr(sys, '_MEIPASS', os.path.dirname(os.path.abspath(__file__)))
    return os.path.join(base_path, relative_path)

def calculate_percentage(value, percentage):
    result = (value * percentage) / 100
    return result

def extracting_numbers_from_text(text_price):
    pattern = r'\d+'
    matches = re.findall(pattern, text_price)
    result = int(''.join(matches))
    return int(result)

pattern = r'\d+'

In [4]:
def open_browser():
    options = webdriver.ChromeOptions()
    options.add_argument('--no--sandbox')
    options.add_argument('no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1080,800')
    options.add_argument('incognito')
    chrome_service = Service('chromedriver')
    chrome_service = Service(executable_path="chromedriver.exe")
    browser = webdriver.Chrome(service=chrome_service, options=options)
    return browser

In [5]:
excel = "C:\\Users\\user\\Project\\FCPI\\회사이름.xlsx"
df = pd.read_excel(excel)
company_names = df['회사이름'].tolist()
company_names = [s.replace(" ", "") for s in company_names]

ceo_names = df['대표자']
ceo_names = [s.replace(" ", "") for s in ceo_names]

In [6]:
browser = open_browser()

all_date_list = []
all_number_list = []
all_company_list = []

for company_name, rep in zip(company_names, ceo_names):
    try:
        browser.get(f"https://www.saramin.co.kr/zf_user/search/company?search_area=main&search_done=y&search_optional_item=n&searchType=search&searchword={company_name}")

        soup = BS(browser.page_source, "html.parser")

        hrefs = []

        corp_infos = soup.find_all(class_="item_corp")

        if len(corp_infos) > 1:
            for corp_info in corp_infos:
                ceo_name_tag = corp_info.find("dt", string="대표자명")

                if ceo_name_tag:
                    ceo_name = ceo_name_tag.find_next("dd").text.strip()

                    if re.search(rep, ceo_name):
                        print(ceo_name)
                        h2_tag = corp_info.find("h2", class_="corp_name")
                        if h2_tag:
                            a_tag = h2_tag.find("a")
                            company_link = a_tag.get("href")
                            full_link = "https://www.saramin.co.kr" + company_link
                            hrefs.append(full_link)

            print(hrefs)
            if len(hrefs) == 1:
                browser.get(hrefs[0])
                time.sleep(.5)

            date_info_list = {}
            numbers_info_list = {}

            personnel_btn = finds_className('btn_age_statistics', browser)
            for idx, _ in enumerate(personnel_btn):
                personnel_btn[idx].click()
                time.sleep(1)

                soup = BS(browser.page_source, "html.parser")
                result = soup.find(class_='graph_range')

                date__info = result.find_all(class_="tit_graph")
                total__info = result.find_all(class_="txt_value")

                counts_list = []
                numbers_list = []

                for da in date__info:
                    counts_list.append(da.text) 

                for nu in total__info:
                    numbers_list.append(nu.text)


                all_company_list.append(find_className('name', browser).text)

                list_name = f"counts_list{idx}"
                date_info_list[list_name] = counts_list

                list2_name = f"numbers_list{idx}"
                numbers_info_list[list2_name] = numbers_list

                all_date_list.append(date_info_list)
                all_number_list.append(numbers_info_list)

        else:
            find_className('company_popup', browser).click()
            time.sleep(.5)

            browser.switch_to.window(browser.window_handles[1])

            date_info_list = {}
            numbers_info_list = {}

            personnel_btn = finds_className('btn_age_statistics', browser)
            for idx, _ in enumerate(personnel_btn):
                personnel_btn[idx].click()
                time.sleep(1)

                soup = BS(browser.page_source, "html.parser")
                result = soup.find(class_='graph_range')

                date__info = result.find_all(class_="tit_graph")
                total__info = result.find_all(class_="txt_value")

                counts_list = []
                numbers_list = []

                for da in date__info:
                    counts_list.append(da.text) 

                for nu in total__info:
                    numbers_list.append(nu.text)


                all_company_list.append(find_className('name', browser).text)

                list_name = f"counts_list{idx}"
                date_info_list[list_name] = counts_list

                list2_name = f"numbers_list{idx}"
                numbers_info_list[list2_name] = numbers_list

                all_date_list.append(date_info_list)
                all_number_list.append(numbers_info_list)

            browser.close()
            browser.switch_to.window(browser.window_handles[0])     

    except Exception as ex:
        pass

김진봉
['https://www.saramin.co.kr/zf_user/company-info/view?csn=WUJOMEhJcE5CWDVtSDNSZjNNY3BjZz09']
최무송
['https://www.saramin.co.kr/zf_user/company-info/view?csn=TkJlcWcwc2NuVi9mYnZBaEZUNlhlZz09']
[]
이종훈
['https://www.saramin.co.kr/zf_user/company-info/view?csn=bW9YdjQwQ2tJQUprWVFXdW9MNDlQdz09']
우상현
['https://www.saramin.co.kr/zf_user/company-info/view?csn=SDIzUVdKNVFPMXpObDFEUzFtMEx5dz09']
조을권
['https://www.saramin.co.kr/zf_user/company-info/view?csn=Q25PYWxMbXowd20wcjEvOUI0NVdKdz09']
주성훈
['https://www.saramin.co.kr/zf_user/company-info/view?csn=VnB3VFVMQlEvODl2c2taM3pCcCtQQT09']
박정식
['https://www.saramin.co.kr/zf_user/company-info/view?csn=VnRwcFIzSGJadStmWGx0Z3dtR3VsZz09']
김사훈
['https://www.saramin.co.kr/zf_user/company-info/view?csn=OTVEUGVLeUFpdFY1VWx6NDZVSE9jZz09']
지창환
['https://www.saramin.co.kr/zf_user/company-info/view?csn=a0xIUnpoS0pqNXZpSDhDS3FzR0J4QT09']
이순묵
['https://www.saramin.co.kr/zf_user/company-info/view?csn=OUN1MFZ1cUxaaGFPSTRGWGllMkp2dz09']
[]
[]
김기영
['https://www.sar

박상만
['https://www.saramin.co.kr/zf_user/company-info/view?csn=VlFkM3ZQaE9XdEIxMnNVQUIzMlMwZz09']
김상철
['https://www.saramin.co.kr/zf_user/company-info/view?csn=cDl2eTdxbWZrMU80bkZIVzNHdTJhUT09']
양승민
['https://www.saramin.co.kr/zf_user/company-info/view?csn=UmtGeUZnTE1ZZkJOc0dEcWJlRVU5QT09']
[]
정승규
['https://www.saramin.co.kr/zf_user/company-info/view?csn=TG5xTDhpT2dlczYvaFM0TkZuZWl2QT09']
송경애
['https://www.saramin.co.kr/zf_user/company-info/view?csn=d3BNckZpNmczYlpmVEVVcjYzek0xdz09']
이재기
['https://www.saramin.co.kr/zf_user/company-info/view?csn=L1BJV0pRb3RCTmQvS1NVazBFNUs2QT09']
[]
정지원
['https://www.saramin.co.kr/zf_user/company-info/view?csn=QWtwcmg0TmFCcjk1WlkzUWJMbUU0Zz09']
김선일/엄정현
['https://www.saramin.co.kr/zf_user/company-info/view?csn=c09mZjhIdXpjOStQcUVQbFFKRXh4QT09']
[]
하규식
['https://www.saramin.co.kr/zf_user/company-info/view?csn=aVBGTEFMbStBeE13OVZ6Znl6bGI3QT09']
안종길
['https://www.saramin.co.kr/zf_user/company-info/view?csn=Zy9JK2t3MkZtRFhxeDVZeHpzMml5QT09']
김기경
['https://www

[]
신광우
['https://www.saramin.co.kr/zf_user/company-info/view?csn=NDZ6cFViOVRTbEVFcnZnem5vOHREQT09']
[]
안광훈
['https://www.saramin.co.kr/zf_user/company-info/view?csn=RXZ5Y3dNYnRFRURBaE0zbmVoL0kzdz09']
[]
박경진
['https://www.saramin.co.kr/zf_user/company-info/view?csn=bmZmL3J0L1kxY0k2cDhMOFYrbEFkZz09']
백규태
['https://www.saramin.co.kr/zf_user/company-info/view?csn=S2pFaDl4NmgvbGw4b0JpTlJZYzcxQT09']
민은기
['https://www.saramin.co.kr/zf_user/company-info/view?csn=bTZtYmlXeUwzSTZkeVlsK3didU1FQT09']
한광택
['https://www.saramin.co.kr/zf_user/company-info/view?csn=QkI3Y3FkMHNSTnR6TldPeTk3QjVldz09']
김호상
['https://www.saramin.co.kr/zf_user/company-info/view?csn=bGR3K3k5OW84ZEUxRjRCYithZEc1UT09']
윤봉훈
['https://www.saramin.co.kr/zf_user/company-info/view?csn=dGlqUUJUaEFMdXV4NW9XYlptM0RPQT09']
조수현
['https://www.saramin.co.kr/zf_user/company-info/view?csn=VUN0amRVWi81TGZzTmN3UkNpd0l2dz09']
안형준
['https://www.saramin.co.kr/zf_user/company-info/view?csn=RW5kSXBIR3RoTkxYVGY4akRxMFpoUT09']
양주섭
['https://www.sar

[]
[]
[]
[]
장기창
['https://www.saramin.co.kr/zf_user/company-info/view?csn=aXB2NVdaMWZQZEtyWjd3L25MRzYvUT09']
이승일
['https://www.saramin.co.kr/zf_user/company-info/view?csn=MXJiTnZWa3RqMXpaYUdYMFZwc1FaQT09']
김영환
김영환
['https://www.saramin.co.kr/zf_user/company-info/view?csn=Z2FzK1F0ZzlMT1lrWmxQNm5rUWpIQT09', 'https://www.saramin.co.kr/zf_user/company-info/view?csn=NCtGSVUvVy9uTFUwZTd0S29GVG9jUT09']
[]
[]
[]
[]
강구학
['https://www.saramin.co.kr/zf_user/company-info/view?csn=b1hmckxDQ2lEdmdObCtFZHRJaGR3QT09']
강민석
['https://www.saramin.co.kr/zf_user/company-info/view?csn=VXkzV3NqdUxSUFBqbzdQeTArM2JnUT09']
[]
이광호
이광호
['https://www.saramin.co.kr/zf_user/company-info/view?csn=MHdKTnc1UDFIYkZvZitCdFYwOEhIZz09', 'https://www.saramin.co.kr/zf_user/company-info/view?csn=Q2lzRHF4b3hyaVFJb0JkbHNsODlGUT09']
정난권
['https://www.saramin.co.kr/zf_user/company-info/view?csn=T2k3MnNsWkhhZVcwNklZdDdsTjRLdz09']
김형섭
['https://www.saramin.co.kr/zf_user/company-info/view?csn=eGxiVExqcFI1a3N4bHNjazloaVhvZz09']
[]
전태

[]
[]
김지인
['https://www.saramin.co.kr/zf_user/company-info/view?csn=ZjlkU0JWckt1dElRcDJTbUJjWERadz09']
전종하
전종하
['https://www.saramin.co.kr/zf_user/company-info/view?csn=eU5JeTEzWmRYZVE4RTlybHBKNzNYQT09', 'https://www.saramin.co.kr/zf_user/company-info/view?csn=V1JKRW1Dc1JoaUd5dnloL0F0a2hnQT09']
김병민
['https://www.saramin.co.kr/zf_user/company-info/view?csn=bFd4MXd0Ylo5eEluMmp6OFFUTUdTUT09']
[]
김가람
['https://www.saramin.co.kr/zf_user/company-info/view?csn=RkNDL1ZFMFBuRTdEOXhLaUsvUkcwUT09']
서종혁
['https://www.saramin.co.kr/zf_user/company-info/view?csn=TGlYbXVGOTcwbm02dFFnaHlMbTlEQT09']
장현국
장현국
장현국
장현국/이길형
장현국
['https://www.saramin.co.kr/zf_user/company-info/view?csn=Rkt1ejR6ZE10azN4QUJTNUJBbUxoQT09', 'https://www.saramin.co.kr/zf_user/company-info/view?csn=bi9NdTlrOWk4a3FWMlpmQWZsVkk5QT09', 'https://www.saramin.co.kr/zf_user/company-info/view?csn=SmhkM2ZFUDZaN2x2cDZpbFBwMkpSZz09', 'https://www.saramin.co.kr/zf_user/company-info/view?csn=YTRxSExZcTRJR25kenExK2NkcG5xUT09', 'https://www.sara

...

서보만
['https://www.saramin.co.kr/zf_user/company-info/view?csn=bnR3Q016Sm40L0xWRGxOQlU4Y0F2QT09']
강인석
['https://www.saramin.co.kr/zf_user/company-info/view?csn=UlM3YnVPTEdRSVJvcEp5OVJYMnA3UT09']
김성진
['https://www.saramin.co.kr/zf_user/company-info/view?csn=bzhZbzE0dFEwY2N2NElWNEkwQlhTZz09']
이원기
['https://www.saramin.co.kr/zf_user/company-info/view?csn=cmF3WnNnelUrRXpKcHU3N0tOM0xpQT09']
송수걸/이기영
['https://www.saramin.co.kr/zf_user/company-info/view?csn=U1E3YWpPNFlmNHdqTVovbDFlNElZdz09']
최주림
['https://www.saramin.co.kr/zf_user/company-info/view?csn=N0dZb0dIcWhqTVlTbGZ5dDk2N2xuZz09']
김태훈
['https://www.saramin.co.kr/zf_user/company-info/view?csn=eXVlSFNDS0VqeUQ0VjVidEFvOGs1dz09']
김영동/김형동
김영동
['https://www.saramin.co.kr/zf_user/company-info/view?csn=cEpoTGN4dU1EZEVBUml1TmVwc3hOZz09', 'https://www.saramin.co.kr/zf_user/company-info/view?csn=TXNHdzFtNXhJSXQyajR0eVNiZjdLQT09']
김효찬
['https://www.saramin.co.kr/zf_user/company-info/view?csn=WWJaUFNoemFRQ0UraVRlR3U2ZFFnQT09']
[]
[]
[]
이종명
['https://

진문화
['https://www.saramin.co.kr/zf_user/company-info/view?csn=aUUrU2d2NlhOemJXTDBHam5IMnlmdz09']
[]
신갑종
['https://www.saramin.co.kr/zf_user/company-info/view?csn=SHFJU0g4bkxsMDdQcUNNK3NWamI3Zz09']
손병규
['https://www.saramin.co.kr/zf_user/company-info/view?csn=aW1Rekpva29GbzhKMXNmbTlZU3BUQT09']
[]
[]
김혁균
['https://www.saramin.co.kr/zf_user/company-info/view?csn=WXgzUXNBUlpqQmhWdzVvR2kvSlJiUT09']
[]
[]
[]
김영빈
김영빈
['https://www.saramin.co.kr/zf_user/company-info/view?csn=VDY5Yzk0TzZoVm1pNEtOOWpKVXBwUT09', 'https://www.saramin.co.kr/zf_user/company-info/view?csn=VnREOFJjSFZlbEtmU0k2blhpeEpvdz09']
[]
정영환
['https://www.saramin.co.kr/zf_user/company-info/view?csn=dG10YlJpUTdqcE5TZUljRExjNHdXQT09']
채백련
['https://www.saramin.co.kr/zf_user/company-info/view?csn=WjV0TG9BZEV4dVRZdlBBcWQ5WEZZQT09']
[]
[]
[]
[]
안태강
안태강
['https://www.saramin.co.kr/zf_user/company-info/view?csn=RHBiRHdEK0ljT1RzdUZzcjRmb3RKZz09', 'https://www.saramin.co.kr/zf_user/company-info/view?csn=NG9HMU9vN2c2Mzh5WTZSdEdMTlB6UT09']

[]
김동훈
['https://www.saramin.co.kr/zf_user/company-info/view?csn=dGJ5SlFZM0FpVzBlRW00WGhBOGU0Zz09']
기창민
기창민
['https://www.saramin.co.kr/zf_user/company-info/view?csn=NGhqOTYxZTE0a3ltcURrODNaUG5Pdz09', 'https://www.saramin.co.kr/zf_user/company-info/view?csn=b0s4QWhDOU11VjdMR2JnQmFaSE10Zz09']
문형준
['https://www.saramin.co.kr/zf_user/company-info/view?csn=MGh3bEhtYkc5U05PU24xVUh1UmNMdz09']
[]
임성욱
['https://www.saramin.co.kr/zf_user/company-info/view?csn=MjV5THgxZHY1OHlsc213MU94VWlaQT09']
김차영
['https://www.saramin.co.kr/zf_user/company-info/view?csn=U0p4alFlQkJocTk1eGw2VEQ5ZmtnQT09']
김윤길
['https://www.saramin.co.kr/zf_user/company-info/view?csn=YjFXMDQyRnROZm5OWlFodURtaDIzQT09']
[]
[]
김재성
['https://www.saramin.co.kr/zf_user/company-info/view?csn=c1B2M2t3aTF2alkzdHFIODFDeC9QQT09']
제이슨사무엘시츠
['https://www.saramin.co.kr/zf_user/company-info/view?csn=c2I2Vmw1eTJxZ2RzalY3elFqU3VBQT09']
박주흠
['https://www.saramin.co.kr/zf_user/company-info/view?csn=ZzgrK1BSVzdSUWQxa2dFYkVjZU04UT09']
김종욱
['https://

[]
유세권
['https://www.saramin.co.kr/zf_user/company-info/view?csn=YzYxcFVwa21nNDIvQ0tGaGdPRUZHZz09']
장용훈
['https://www.saramin.co.kr/zf_user/company-info/view?csn=L1BVZ2o0OVU2WlBIRVNmY2RlY3hCQT09']
김희민
['https://www.saramin.co.kr/zf_user/company-info/view?csn=WlU2RldxOFJwN0ZMZFR2NjdSVXloQT09']
오일택
['https://www.saramin.co.kr/zf_user/company-info/view?csn=NE01dmUyMnhFVVZ3Skg3N3BOTkdlUT09']
조영근
['https://www.saramin.co.kr/zf_user/company-info/view?csn=elJNOWhuNTQ1YUV5dHdudDZPYVoxZz09']
[]
박운진
['https://www.saramin.co.kr/zf_user/company-info/view?csn=dTdodWdKaWFYU3BvSHphbndCSGhYZz09']
[]
김진희
['https://www.saramin.co.kr/zf_user/company-info/view?csn=dzZRQVh4MGRaZDJrTU5tS1VNY2FvQT09']
권영찬
['https://www.saramin.co.kr/zf_user/company-info/view?csn=Z1RmUlZzKzc2L0J2TVN0K2IveXR5Zz09']
[]
[]
양태진
['https://www.saramin.co.kr/zf_user/company-info/view?csn=RkUwVlNxNlJOQVJWZHEyK2p0cE1YZz09']
김정석
['https://www.saramin.co.kr/zf_user/company-info/view?csn=bDRiYnpBczZOZkpSUGJ4LytQZWxVUT09']
장종훈
['https://w

최영건
['https://www.saramin.co.kr/zf_user/company-info/view?csn=a2kySFR5MmlKTjlPRytSQUdVNGp5Zz09']
[]
진민호
['https://www.saramin.co.kr/zf_user/company-info/view?csn=c2x3UGpsL0xOOW1nbDR0dnYzSmxmQT09']
김차영
['https://www.saramin.co.kr/zf_user/company-info/view?csn=blFlVVdEZmd3UUFXZm1QVnI1K0k3QT09']
백승환
['https://www.saramin.co.kr/zf_user/company-info/view?csn=UTlpVGxJTTNBSDdTcHRYcUJkaGl4dz09']
[]
노태윤
['https://www.saramin.co.kr/zf_user/company-info/view?csn=bmdDRi9DUVlFeXI0b2tQWE5uVmtOZz09']
[]
신현숙
['https://www.saramin.co.kr/zf_user/company-info/view?csn=NUlnSE45RTJyYWVRcTNuSFB2OTViUT09']
박근만
['https://www.saramin.co.kr/zf_user/company-info/view?csn=dWJEaC92dnVpZ3ZoMmJXTjVUWDFOdz09']
[]
[]
[]
임형철
['https://www.saramin.co.kr/zf_user/company-info/view?csn=Wm9JTzZLZDIzVnkxUE50WXZQdmhjZz09']
[]
김동현
['https://www.saramin.co.kr/zf_user/company-info/view?csn=dnF6WThEOEpYM3RBK29QYXRHUDd2UT09']
진환구
['https://www.saramin.co.kr/zf_user/company-info/view?csn=UXVGakdId0VydzdxaDNoVXZQQ29mUT09']
조재준
['htt

In [7]:
print(len(all_date_list))
print(len(all_number_list))
print(len(all_company_list))

10968
10968
10968


In [8]:
df = pd.DataFrame({
    "company_name" : all_company_list,
    "date" : all_date_list,
    "numbers" : all_number_list
})

In [9]:
df.duplicated(['company_name']).sum()

8278

In [10]:
df2 = df.copy()
df2.head()

,company_name,date,numbers
0,(주)피피아이,"{'counts_list0': ['기술직', '연구직', '생산직', '사무직'],...",{'numbers_list0': [' ...
1,(주)피피아이,"{'counts_list0': ['기술직', '연구직', '생산직', '사무직'],...",{'numbers_list0': [' ...
2,(주)피피아이,"{'counts_list0': ['기술직', '연구직', '생산직', '사무직'],...",{'numbers_list0': [' ...
3,(주)자람테크놀로지,"{'counts_list0': ['전사'], 'counts_list1': ['전사'...",{'numbers_list0': [' ...
4,(주)자람테크놀로지,"{'counts_list0': ['전사'], 'counts_list1': ['전사'...",{'numbers_list0': [' ...


In [11]:
df2.drop_duplicates(['company_name'], inplace=True)

In [12]:
df2.duplicated(['company_name']).sum()

0

In [13]:
df2.shape

(2690, 3)

In [14]:
df2.to_excel("company_personnel_info2.xlsx", index=False)

## Data Preprocessing

In [16]:
excel2 = "C:\\Users\\user\\Project\\FCPI\\company_personnel_info2.xlsx"
data = pd.read_excel(excel2)
data2 = data.copy()
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2690 entries, 0 to 2689
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_name  2690 non-null   object
 1   date          2690 non-null   object
 2   numbers       2690 non-null   object
dtypes: object(3)
memory usage: 63.2+ KB


In [19]:
data2['date'][3]

"{'counts_list0': ['2023.5', '2023.6', '2023.7', '2023.8', '2023.9'], 'counts_list1': ['2023.5', '2023.6', '2023.7', '2023.8', '2023.9'], 'counts_list2': ['2023.5', '2023.6', '2023.7', '2023.8', '2023.9']}"

In [20]:
final_company_list = []
final_date_list = []
final_total_numbers_list = []
final_joined_numbers_list = []
final_leaves_numbers_list = []

for i in range(len(data2['date'])):
    data_dict = ast.literal_eval(data2['date'][i])
    data_dict2 = ast.literal_eval(data2['numbers'][i])

    counts_list0 = data_dict.get('counts_list0', [])
    counts_list1 = data_dict.get('counts_list1', [])
    counts_list2 = data_dict.get('counts_list2', [])

    numbers_list0 = data_dict2.get('numbers_list0', [])
    numbers_list1 = data_dict2.get('numbers_list1', [])
    numbers_list2 = data_dict2.get('numbers_list2', [])
    
    final_company_list.append(data2['company_name'][i])
    final_date_list.append(counts_list0[-1])
    
    final_total_numbers_list.append(numbers_list0[-1])
    final_joined_numbers_list.append(numbers_list1[-1])
    final_leaves_numbers_list.append(numbers_list2[-1])
    print("총 인원 수 날짜 :",counts_list0[-1], "/ 총 인원 수 :",numbers_list0[-1])
    print("입사자 날짜 :",counts_list1[-1], "/ 입사자 수 :",numbers_list1[-1])
    print("퇴사자 날짜 :",counts_list2[-1], "/ 퇴사자 수 :",numbers_list2[-1])
    print("\n")

총 인원 수 날짜 : 사무직 / 총 인원 수 : 
                                        5년 1개월                                    
입사자 날짜 : 사무직 / 입사자 수 : 
                                        5년 1개월                                    
퇴사자 날짜 : 사무직 / 퇴사자 수 : 
                                        5년 1개월                                    


총 인원 수 날짜 : 전사 / 총 인원 수 : 
                                        6년 3개월                                    
입사자 날짜 : 전사 / 입사자 수 : 
                                        6년 3개월                                    
퇴사자 날짜 : 전사 / 퇴사자 수 : 
                                        6년 3개월                                    


총 인원 수 날짜 : 2023.9 / 총 인원 수 : 42
입사자 날짜 : 2023.9 / 입사자 수 : 0
퇴사자 날짜 : 2023.9 / 퇴사자 수 : 1


총 인원 수 날짜 : 2023.9 / 총 인원 수 : 26
입사자 날짜 : 2023.9 / 입사자 수 : 0
퇴사자 날짜 : 2023.9 / 퇴사자 수 : 1


총 인원 수 날짜 : 전사 / 총 인원 수 : 
                                        7년                                    
입사자 날짜 : 전사 / 입사자 수 : 
                                      

총 인원 수 날짜 : 2023.9 / 총 인원 수 : 45
입사자 날짜 : 2023.9 / 입사자 수 : 1
퇴사자 날짜 : 2023.9 / 퇴사자 수 : 1


총 인원 수 날짜 : 2023.9 / 총 인원 수 : 37
입사자 날짜 : 2023.9 / 입사자 수 : 0
퇴사자 날짜 : 2023.9 / 퇴사자 수 : 0


총 인원 수 날짜 : 2023.9 / 총 인원 수 : 84
입사자 날짜 : 2023.9 / 입사자 수 : 0
퇴사자 날짜 : 2023.9 / 퇴사자 수 : 1


총 인원 수 날짜 : 회사전체 / 총 인원 수 : 
                                        2년 9개월                                    
입사자 날짜 : 회사전체 / 입사자 수 : 
                                        2년 9개월                                    
퇴사자 날짜 : 회사전체 / 퇴사자 수 : 
                                        2년 9개월                                    


총 인원 수 날짜 : 2023.9 / 총 인원 수 : 37
입사자 날짜 : 2023.9 / 입사자 수 : 1
퇴사자 날짜 : 2023.9 / 퇴사자 수 : 0


총 인원 수 날짜 : 2023.9 / 총 인원 수 : 57
입사자 날짜 : 2023.9 / 입사자 수 : 0
퇴사자 날짜 : 2023.9 / 퇴사자 수 : 0


총 인원 수 날짜 : 2023.9 / 총 인원 수 : 44
입사자 날짜 : 2023.9 / 입사자 수 : 0
퇴사자 날짜 : 2023.9 / 퇴사자 수 : 0


총 인원 수 날짜 : 2023.9 / 총 인원 수 : 40
입사자 날짜 : 2023.9 / 입사자 수 : 1
퇴사자 날짜 : 2023.9 / 퇴사자 수 : 1


총 인원 수 날짜 : 2023.9 / 총 인원 수 : 25


In [21]:
print(len(final_company_list))
print(len(final_date_list))
print(len(final_total_numbers_list))
print(len(final_joined_numbers_list))
print(len(final_leaves_numbers_list))

2690
2690
2690
2690
2690


In [22]:
final_data = pd.DataFrame({
    "회사이름" : final_company_list,
    "날짜" : final_date_list,
    "총 인원수" : final_total_numbers_list,
    "입사자 수" : final_joined_numbers_list,
    "퇴사자 수" : final_leaves_numbers_list
})

final_data

,회사이름,날짜,총 인원수,입사자 수,퇴사자 수
0,(주)피피아이,사무직,\n 5년 1...,\n 5년 1...,\n 5년 1...
1,(주)자람테크놀로지,전사,\n 6년 3...,\n 6년 3...,\n 6년 3...
2,엠에스테크놀러지(주),2023.9,42,0,1
3,(주)스프링웨이브,2023.9,26,0,1
4,현대에이치티(주),전사,\n 7년 ...,\n 7년 ...,\n 7년 ...
...,...,...,...,...,...
2685,(주)디에스티인터내셔날,2023.9,130,3,1
2686,(주)씨엠비광주방송,2023.9,26,1,0
2687,(주)씨엠비대구방송,2023.1,14,0,0
2688,(주)씨엠비영등포방송,2023.9,30,1,0


In [23]:
final_data.to_excel("ICT회사 원표본 회사 인원 데이터.xlsx", index=False)